# Portafolios de Inversión
## Tarea 7

Presenta : 

- José Armando Melchor Soto


---
### Librerías 



In [34]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize 

---

#### Indicaciones 

1.- Función de Métricas de Desempeño. (20%)

Genera una función en python que reciba un data frame con la evolución histórica de n cantidad de estrategías de inversión y calcule todas las métricas de desempeño vistas en las dos clases anteriores (Rendimiento, Volatilidad, Ratio de Sharpe, Downside, Upside, Omega, Beta, Alfa de Jensen, Ratio de Treynor, Ratio de Sortino).
 

2.- Backtesting de Portafolio. (50%)

Asume que quieres invertir en un portafolio compuesto por las siguientes acciones mexicanas: LIVEPOLC1.MX, KOFUBL.MX, CEMEXCPO.MX, CHDRAUIB.MX, CUERVO.MX, LABB.MX, RA.MX.
Debido a que no sabes como ponderarlos, decides realizar el backtesting para el portafolio para cuatro estrategias de Asset Allocation: Mínima Varianza, Máximo de Sharpe, Semivarianza Target y Omega.
Realiza el backtesting utilizando datos para simulación del 23-06-2024 hasta el 23-06-2025, toma un período anterior con el mismo horizonte temporal para optimizar las ponderaciones.
Recuerda siempre comparar con una estrategia de inversión pasiva, en este caso la opción lógica es el IPC México.
Gráfica la evolución histórica de las cinco alternativas de inversión.
Con los resultados del backtesting histórico calcula e interpreta las métricas de desempeño para las cinco alternativas de estrategias de inversión simuladas.
 

3.- Selección de Estrategia de Inversión. (20%)

Utilizando los resultados de tu backtesting y las métricas de desempeño, selecciona una estrategia de inversión. Realiza una conclusión clara, breve y robusta de porque es la mejor estrategia PARA TI, apoyate en las métricas obtenidas y en tu perfil de aversión al riesgo.
 

4.- Calculo de Pesos para Portafolio Real. (10%)

En base a la estrategia de Asset Allocation seleccionada en el punto anterior, calcula los pesos eficientes y la cantidad de acciones que utilizarías para invertir en la realidad en tu portafolio. Recuerda tomar la ventana de datos más reciente, en función del horizonte temporal tomado. 

---

#### Funciones 

##### Función Mínima Varianza 

In [35]:
def opt_min_var(rets):
    
    cov=rets.cov()
    n_assets = len(rets.columns)
    
    var = lambda w: w.T @ cov @ w
    
    w0=np.ones(n_assets)/n_assets
    
    bounds=[(0, 1)]*n_assets
    
    constraint=lambda w: sum(w)-1
    
    result=minimize(fun=var, x0=w0, bounds=bounds, constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)
    
    return result.x

##### Función Máximo de Sharpe

In [36]:
def opt_max_sharpe(rets, rf):
    
    E=rets.mean()
    cov=rets.cov()
    n_assets = len(rets.columns)
    
    rs = lambda w: -((w.T @ E - rf/252) / np.sqrt(w.T @ cov @ w))
    
    w0=np.ones(n_assets)/n_assets
    
    bounds=[(0, 1)]*n_assets
    
    constraint=lambda w: sum(w)-1
    
    result=minimize(fun=rs, x0=w0, bounds=bounds, constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)
    
    return result.x

##### Función Semi-Varianza Target

In [37]:
def objetivo (rets:pd.DataFrame , rets_b : pd.DataFrame ,corr:pd.DataFrame):

    diffs=rets - rets_b.values
    rends_below_bench = diffs[diffs < 0].fillna(0)
    target_downside_risk=np.array(rends_below_bench.std())
    target_semivarmatrix= corr * (target_downside_risk.reshape(len(rets.columns), 1) @ target_downside_risk.reshape(1, len(rets.columns)))

    objectivo = lambda w: (w.T @ target_semivarmatrix @ w)

    n = len(rets.keys())
    w_inicial = np.ones(n)/n
    bounds = [(0,1)]*n
    tol = 1e-9
    rest = lambda w: np.sum(w) - 1
    
    obj = minimize(
    fun=objectivo,
    x0=w_inicial,
    bounds=bounds,
    constraints={'fun': rest, 'type': 'eq'},
    tol=tol)

    w_objetivo = obj.x

    return dict(zip(rets.columns, w_objetivo))

##### Función Omega


In [38]:
def scipy_omega(rets: pd.DataFrame):
    n = len(rets.columns)
    w_inicial = np.ones(n) / n
    bounds = [(0, 1)] * n
    tol = 1e-16

    rets_below_zero = rets[rets < 0].fillna(0)
    rets_above_zero = rets[rets > 0].fillna(0)
    upside_risk=rets_above_zero.std()
    downside_risk = rets_below_zero.std()

    omega_ind = upside_risk / downside_risk

    Omega_neg = lambda w: -np.sum(w * omega_ind)

    constraints = {'fun': lambda w: np.sum(w) - 1, 'type': 'eq'}

    result = minimize(
        fun=Omega_neg,
        x0=w_inicial,
        bounds=bounds,
        constraints=constraints,
        tol=tol
    )

    w_omega = result.x
    return dict(zip(rets.columns, w_omega))


##### Función Backtesting 

In [39]:
def backtesting(prices: pd.DataFrame, benchmark: pd.DataFrame, weights: dict, capital: float):
    
    # Obtener rendimientos de precios y benchmark
    rets_assets = prices.pct_change().dropna()
    rets_benchmark = benchmark.pct_change().dropna()
    
    # data frame vacio para llenar historia
    
    df = pd.DataFrame(index=rets_assets.index)
    
    # Obtener evolución para cada estrategia dada
    for strategy in weights.keys():
        # para benchmark
        if strategy == 'benchmark':
            temp = rets_benchmark + 1
            temp.iloc[0]=capital
            temp = temp.cumprod()
            df[strategy] = temp.values            
    
        # para otras estrategias
    
        else:
            temp = (rets_assets * weights.get(strategy)).sum(axis=1)+1
            temp.iloc[0]=capital
            temp = temp.cumprod()
            df[strategy] = temp.values    

    # Graficar la evolucion para ambos portafolios
    plt.figure(figsize=(12,6))
    plt.plot(df, label=df.keys())
    plt.title('Backtesting of Strategies')
    plt.xlabel('Date')
    plt.ylabel('Portafolio Value')
    plt.legend()
    plt.show()


    return df

##### Función Métricas de Desempeño

In [40]:
def metricas_anuales(history, rf):
    # Returns
    rets= history.pct_change().dropna()
    
    # Matriz de Cov
    cov_hist = rets.cov()

    # Dataframe
    metricas = pd.DataFrame(index=rets.columns)

    # Rendimiento 
    metricas['Rendimiento Promedio Anual'] = R_p = rets.mean()*252*100
    
    # Volatilidad 
    metricas['Volatilidad Anual'] = vol_anual = rets.std()*np.sqrt(252)*100
    
    # Ratio de Sharpe
    metricas['Ratio de Sharpe'] = (R_p - rf) / vol_anual

    # Downside Risk
    metricas['Downside Risk'] = downside = rets[rets < 0].std() * np.sqrt(252) * 100

    # Upside Risk
    metricas['Upside Risk'] = upside = rets[rets > 0].std() * np.sqrt(252) * 100

    # Omega
    metricas['Omega'] =omega= upside / downside

    # Beta

    covs = cov_hist['Benchmark']
    varianza_market = covs['Benchmark']
    metricas['Beta'] = beta = covs / varianza_market

    # Alfa de Jensen

    R_m = rets['Benchmark']
    alpha_Jensen = R_p - (rf + beta * (R_m - rf))

    # Ratio de Treynor
    metricas['Ratio de Treynor'] = (R_p - rf) / beta

    # Ratio de Sortino
    metricas['Ratio de Sortino'] = (R_p - rf) / downside


---

#### Construcción del Portafolio 

##### Activos 

| **Empresa**                             | **Ticker**        | **Sector Económico**                          |
|----------------------------------------|-------------------|-----------------------------------------------|
| El Puerto de Liverpool, S.A.B. de C.V. | LIVEPOLC-1.MX     | Consumo                 |
| Coca-Cola FEMSA, S.A.B. de C.V.        | KOFUBL.MX         | Consumo Básico             |
| Cemex, S.A.B. de C.V.                  | CEMEXCPO.MX       | Materiales           |
| Grupo Comercial Chedraui               | CHDRAUIB.MX       | Consumo Básico          |
| Becle, S.A.B. de C.V. (José Cuervo)    | CUERVO.MX         | Consumo Básico       |
| Genomma Lab Internacional, S.A.B. de C.V. | LABB.MX         | Salud  |
| Rassini, S.A.B. de C.V.                | RA.MX             | Industriales                   |



##### Benchmark

##### Sector económico :   

- Vanguard Consumer Staples Index Fund ETF Shares (VDC)

---

#### Importación de los datos

In [41]:
tickers = ['LIVEPOLC-1.MX', 'KOFUBL.MX', 'CEMEXCPO.MX', 'CHDRAUIB.MX', 'CUERVO.MX', 'LABB.MX', 'RA.MX']

opt_prices = yf.download(tickers, start='2023-06-23', end='2024-06-23', progress=False)['Close']

bt_prices = yf.download(tickers, start='2024-06-23', end='2025-06-23', progress=False)['Close']

benchmark = yf.download('^MXX', start='2024-06-23', end='2025-06-23', progress=False)['Close']


1 Failed download:
['LABB.MX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-06-23 -> 2025-06-23)')


In [42]:
opt_rets = opt_prices.pct_change().dropna()
corr = opt_rets.corr()

---

#### Asset Allocation 

##### Mínima Varianza

In [43]:
w_minvar = opt_min_var(opt_rets)
dict(zip(opt_rets.columns, w_minvar))

{'CEMEXCPO.MX': 0.046133541750810476,
 'CHDRAUIB.MX': 0.30909671034200686,
 'CUERVO.MX': 0.1171090178446094,
 'KOFUBL.MX': 0.21646726609771685,
 'LABB.MX': 0.0014443505586861209,
 'LIVEPOLC-1.MX': 0.24720922954684668,
 'RA.MX': 0.06253988385932366}

##### Máximo de Sharpe 

In [44]:
w_maxsharpe = opt_max_sharpe(opt_rets, rf=0.045)
dict(zip(opt_rets.columns, w_maxsharpe))

{'CEMEXCPO.MX': 0.0,
 'CHDRAUIB.MX': 0.5377960782104608,
 'CUERVO.MX': 0.0,
 'KOFUBL.MX': 1.8952129691044473e-17,
 'LABB.MX': 0.09006442357229631,
 'LIVEPOLC-1.MX': 0.3721394982172429,
 'RA.MX': 2.114614234845576e-18}

##### Semi-Varianza target

##### Ratio Omega

In [45]:
omega = scipy_omega(opt_rets)
omega

{'CEMEXCPO.MX': 1.2484423966668938e-16,
 'CHDRAUIB.MX': 0.9999999999999998,
 'CUERVO.MX': 2.52395548196098e-17,
 'KOFUBL.MX': 2.051302103659791e-16,
 'LABB.MX': 1.0272992762106022e-16,
 'LIVEPOLC-1.MX': 0.0,
 'RA.MX': 5.943376018590351e-18}

---

#### Backtesting de Portafolio

##### Capital 

In [46]:
capital = 1000000

##### Estrategias a usar

In [ ]:
w_dict = {
    'Min Varianza': w_minvar,
    'Max Sharpe': w_maxsharpe,
    'Semi-Varianza' : w_semivar,
    'Ratio Omega': omega,
    'Benchmark': 1
}

##### Función Aplicada 

In [50]:
history = backtesting(bt_prices, benchmark, w_dict, capital)

C:\Users\ppmel\AppData\Local\Temp\ipykernel_22016\2574611907.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets_assets = prices.pct_change().dropna()


IndexError: iloc cannot enlarge its target object

---